In [1]:
import tensorflow as tf 
from tensorflow import keras
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt 
import time

In [3]:
df = pd.read_csv(r'train.csv')

In [4]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df.shape

(7613, 5)

In [6]:
print((df.target == 1).sum()) # disaster
print((df.target == 0).sum()) # no disaster

3271
4342


In [12]:
import re  # Regular Expression
import string

def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_punc(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

# Example usage:
sample_text = "Check this out: https://example.com!!!"
print(remove_url(sample_text))  # Removes URL
print(remove_punc(sample_text)) # Removes punctuation



Check this out: 
Check this out httpsexamplecom


In [13]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")
for t in df.text:
    matches = pattern.findall(t)
    for match in matches:
        print("1")
        print(t)
        print("2")
        print(match)
        print("3")
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

1
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
2
t
3
@bbcmtd Wholesale Markets ablaze 


In [14]:
df['text'] = df.text.map(remove_url)        # map(lambda x: remove_url(x))
df['text'] = df.text.map(remove_punc)
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1
2,5,NaN,NaN,All residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1


In [15]:
# remove stopwords 
# (Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored           without sacrificing the meaning of the sentence. For example, the words like the, he, have etc. Such words are      already captured this in corpus named corpus.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kirthana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [16]:
df["text"] = df.text.map(remove_stopwords)
df.text

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [17]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

counter = counter_word(df.text)

In [18]:
len(counter)

17971

In [19]:
# counter

In [20]:
counter.most_common(5)

[('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]

In [21]:
num_unique_words = len(counter)

In [22]:
train_size = int(df.shape[0] * 0.8)
train_df = df[:train_size]
val_df = df[train_size:]

train_sentence = train_df.text.to_numpy()
train_labels = train_df.target.to_numpy()
val_sentence = val_df.text.to_numpy()
val_labels = val_df.target.to_numpy()

In [23]:
train_sentence.shape, val_sentence.shape
train_sentence

array(['deeds reason earthquake may allah forgive us',
       'forest fire near la ronge sask canada',
       'residents asked shelter place notified officers evacuation shelter place orders expected',
       ..., 'feel like sinking unhappiness take quiz',
       'sinking music video tv career brooke hogan thanking dad free publicityalthough doubt help',
       'supernovalester feel bad literally feel feeling heart sinking bc didnt get anyone ugh jfc'],
      dtype=object)

In [24]:
# tokenize 

from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of interger
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentence)  # fit only to training

In [25]:
# each word has unique index
word_index = tokenizer.word_index

In [26]:
# word_index

In [27]:
train_sequence = tokenizer.texts_to_sequences(train_sentence)
val_sequence = tokenizer.texts_to_sequences(val_sentence)

In [28]:
print(train_sentence[0:5])
print(train_sequence[0:5])

['deeds reason earthquake may allah forgive us'
 'forest fire near la ronge sask canada'
 'residents asked shelter place notified officers evacuation shelter place orders expected'
 '13000 people receive wildfires evacuation orders california'
 'got sent photo ruby alaska smoke wildfires pours school']
[[3739, 696, 235, 41, 1282, 3740, 14], [71, 3, 129, 576, 5670, 5671, 1283], [1448, 1186, 1882, 495, 5672, 1449, 116, 1882, 495, 976, 1187], [2243, 8, 3741, 1070, 116, 976, 24], [27, 1071, 358, 5673, 1635, 892, 1070, 5674, 91]]


In [29]:
# pad the sentence to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Maximum number of words in a suquence
max_length = 20

train_padded = pad_sequences(train_sequence, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequence, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((6090, 20), (1523, 20))

In [30]:
train_padded[0]

array([3739,  696,  235,   41, 1282, 3740,   14,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [31]:
print(train_sentence[10])
print(train_sequence[10])
print(train_padded[10])

three people died heat wave far
[520, 8, 395, 156, 297, 411]
[520   8 395 156 297 411   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]


In [32]:
# Check reversing the indices

# flip key, values
reverse_word_index = dict([(idx, word) for word, idx in word_index.items()])

In [33]:
# reverse_word_index

In [34]:
def decode(sequence):
    return ' '.join([reverse_word_index.get(idx, "?") for idx in sequence])

In [35]:
decode_text = decode(train_sequence[10])
print(train_sequence[10])
print(decode_text)

[520, 8, 395, 156, 297, 411]
three people died heat wave far


In [36]:
# Create LSTM Model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length)
# and  the largest integer (i.e. word index) in the input should be no longer than num_words (vocabulary size)
# Now model.output_shape is (None, input_length, 32), where None is tha batch dimension

model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

C:\Users\Kirthana\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optim, metrics=metrics)


In [41]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/20
191/191 - 10s - 51ms/step - accuracy: 0.7136 - loss: 0.5509 - val_accuracy: 0.7702 - val_loss: 0.4829
Epoch 2/20
191/191 - 4s - 19ms/step - accuracy: 0.8864 - loss: 0.2952 - val_accuracy: 0.7623 - val_loss: 0.5374
Epoch 3/20
191/191 - 4s - 19ms/step - accuracy: 0.9445 - loss: 0.1679 - val_accuracy: 0.7669 - val_loss: 0.6979
Epoch 4/20
191/191 - 4s - 19ms/step - accuracy: 0.9644 - loss: 0.1152 - val_accuracy: 0.7676 - val_loss: 0.6781
Epoch 5/20
191/191 - 4s - 19ms/step - accuracy: 0.9713 - loss: 0.0949 - val_accuracy: 0.7393 - val_loss: 0.6771
Epoch 6/20
191/191 - 4s - 19ms/step - accuracy: 0.9754 - loss: 0.0764 - val_accuracy: 0.7498 - val_loss: 0.7571
Epoch 7/20
191/191 - 6s - 30ms/step - accuracy: 0.9777 - loss: 0.0728 - val_accuracy: 0.7393 - val_loss: 0.7410
Epoch 8/20
191/191 - 4s - 20ms/step - accuracy: 0.9785 - loss: 0.0637 - val_accuracy: 0.7472 - val_loss: 0.7654
Epoch 9/20
191/191 - 4s - 19ms/step - accuracy: 0.9788 - loss: 0.0529 - val_accuracy: 0.7163 - val_loss

In [42]:
predictions = model.predict(train_padded)
predictions = [1 if p> 0.5 else 0 for p in predictions]

191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [43]:
print(train_sentence[10:20])

['three people died heat wave far'
 'haha south tampa getting flooded hah wait second live south tampa gonna gonna fvck flooding'
 'raining flooding florida tampabay tampa 18 19 days ive lost count'
 'flood bago myanmar arrived bago'
 'damage school bus 80 multi car crash breaking' 'whats man' 'love fruits'
 'summer lovely' 'car fast' 'goooooooaaaaaal']


In [44]:
print(train_labels[10:20])
print(predictions[10:20])

[1 1 1 1 1 0 0 0 0 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
